In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
!pip install anvil-uplink

  Using cached argparse-1.4.0-py2.py3-none-any.whl.metadata (2.8 kB)
Using cached argparse-1.4.0-py2.py3-none-any.whl (23 kB)


In [3]:
import anvil.server
anvil.server.connect("server_ACQH7UGAX7FKFZYLBI3THIFY-VCC5GNK5MLX2QFVD")

Connecting to wss://anvil.works/uplink
Anvil websocket open
Connected to "Default Environment" as SERVER


In [4]:
# pip install tensorflow keras pickle nltk
from google.colab import drive
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
# from keras.optimizers import SGD
from tensorflow.keras.optimizers import SGD
import random
import nltk
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
import json
import pickle
intents_file = open('/content/drive/MyDrive/intents.json').read()
intents = json.loads(intents_file)

In [5]:
import nltk
nltk.download('punkt')
words=[]
classes = []
documents = []
ignore_letters = ['!', '?', ',', '.']
for intent in intents['intents']:
    for pattern in intent['patterns']:
        #tokenize each word
        word = nltk.word_tokenize(pattern)
        words.extend(word)
        #add documents in the corpus
        documents.append((word, intent['tag']))
        # add to our classes list
        if intent['tag'] not in classes:
            classes.append(intent['tag'])
print(documents)

[(['hi'], 'greeting'), (['hello'], 'greeting'), (['whats', 'up'], 'greeting'), (['sup'], 'greeting'), (['is', 'anyone', 'there'], 'greeting'), (['whats', 'good'], 'greeting'), (['hey'], 'greeting'), (['bye'], 'goodbye'), (['cya'], 'goodbye'), (['see', 'you', 'later'], 'goodbye'), (['goodbye'], 'goodbye'), (['im', 'leaving'], 'goodbye'), (['have', 'a', 'good', 'day'], 'goodbye'), (['how', 'old', 'are', 'you'], 'age'), (['what', 'is', 'your', 'age'], 'age'), (['thanks'], 'thanks'), (['thank', 'you'], 'thanks'), (['thankyou'], 'thanks'), (['ty'], 'thanks'), (['I', 'owe', 'you', 'one'], 'thanks'), (['whats', 'is', 'your', 'name'], 'name'), (['whats', 'your', 'name'], 'name'), (['whats', 'should', 'I', 'call', 'you'], 'name'), (['how', 'should', 'I', 'address', 'you'], 'name'), (['no'], 'sky_net_no'), (['nah'], 'sky_net_no'), (['not', 'really'], 'sky_net_no'), (['thats', 'scary'], 'sky_net_no'), (['singularity'], 'sky_net_no'), (['how', 'are', 'you'], 'how_are_you'), (['how', 'are', 'you', 

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [6]:
import nltk
nltk.download('wordnet')
# create the training data
training = []
# create empty array for the output
output_empty = [0] * len(classes)
# training set, bag of words for every sentence
for doc in documents:
    # initializing bag of words

    bag = []
    # list of tokenized words for the pattern
    word_patterns = doc[0]
    # lemmatize each word - create base word, in attempt to represent related words
    word_patterns = [lemmatizer.lemmatize(word.lower()) for word in word_patterns]
    # create the bag of words array with 1, if word is found in current pattern
    for word in words:
        bag.append(1) if word in word_patterns else bag.append(0)
    # output is a '0' for each tag and '1' for current tag (for each pattern)
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1

    # Pad the shorter list with zeros
    if len(bag) < len(output_row):
        bag += [0] * (len(output_row) - len(bag))
    elif len(output_row) < len(bag):
        output_row += [0] * (len(bag) - len(output_row))
    training.append([bag, output_row])

# shuffle the features and make numpy array
random.shuffle(training)
training = np.array(training)
# create training and testing lists. X - patterns, Y - intents
train_x = list(training[:,0])
train_y = list(training[:,1])
print("Training data is created")

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


Training data is created


In [10]:
# deep neural networds model
model = Sequential()
model.add(Dense(128, input_shape=(len(train_x[0]),), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation='softmax'))
# Compiling model. SGD with Nesterov accelerated gradient gives good results for this model
sgd = SGD(learning_rate=0.01,momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])
#Training and saving the model
hist = model.fit(np.array(train_x), np.array(train_y), epochs=200, batch_size=5, verbose=1)
model.save('chatbot_model.h5', hist)
print("model is created")

Epoch 1/200


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


20/20 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.0112 - loss: 5.3855    
Epoch 2/200
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.1718 - loss: 5.0967  
Epoch 3/200
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.1958 - loss: 4.4278     
Epoch 4/200
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.1589 - loss: 3.9563 
Epoch 5/200
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.2733 - loss: 3.3248  
Epoch 6/200
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.2517 - loss: 2.7224 
Epoch 7/200
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.3384 - loss: 2.5092 
Epoch 8/200
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.2194 - loss: 2.4667      
Epoch 9/200
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.1476 - loss: 2.4718     
Epoch 10/200
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.2514 - loss: 2.3847     
Epoch 11/200
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.3226 - loss: 2.1441  
Epoch 12/200
20/20 ━━━━━━━━━━━━━━━━━━━━ 0

model is created


In [11]:
import nltk
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
import pickle
import numpy as np
from keras.models import load_model
model = load_model('chatbot_model.h5')
import json
import random
intents = json.loads(open('/content/drive/MyDrive/intents.json').read())
words = pickle.load(open('/content/drive/MyDrive/words.pkl','rb'))
classes = pickle.load(open('/content/drive/MyDrive/classes.pkl','rb'))
def clean_up_sentence(sentence):
    # tokenize the pattern - splitting words into array
    sentence_words = nltk.word_tokenize(sentence)
    # stemming every word - reducing to base form
    sentence_words = [lemmatizer.lemmatize(word.lower()) for word in sentence_words]
    return sentence_words
# return bag of words array: 0 or 1 for words that exist in sentence
def bag_of_words(sentence, words, show_details=True):
    # tokenizing patterns
    sentence_words = clean_up_sentence(sentence)
    # bag of words - vocabulary matrix
    bag = [0]*len(words)
    for s in sentence_words:
        for i,word in enumerate(words):
            if word == s:
                # assign 1 if current word is in the vocabulary position
                bag[i] = 1
                if show_details:
                    print ("found in bag: %s" % word)
    return(np.array(bag))
def predict_class(sentence):
    # filter below  threshold predictions
    p = bag_of_words(sentence, words,show_details=False)
    res = model.predict(np.array([p]))[0]
    ERROR_THRESHOLD = 0.25
    results = [[i,r] for i,r in enumerate(res) if r>ERROR_THRESHOLD]
    # sorting strength probability
    results.sort(key=lambda x: x[1], reverse=True)
    return_list = []
    for r in results:
        return_list.append({"intent": classes[r[0]], "probability": str(r[1])})
    return return_list
def getResponse(ints, intents_json):
    tag = ints[0]['intent']
    list_of_intents = intents_json['intents']
    for i in list_of_intents:
        if(i['tag']== tag):
            result = random.choice(i['responses'])
            break
    return result
#Creating tkinter GUI
# import tkinter
# from tkinter import *
# def send():
#     msg = EntryBox.get("1.0",'end-1c').strip()
#     EntryBox.delete("0.0",END)
#     if msg != '':
#         ChatBox.config(state=NORMAL)
#         ChatBox.insert(END, "You: " + msg + '\n\n')
#         ChatBox.config(foreground="#446665", font=("Verdana", 12 ))
#         ints = predict_class(msg)
#         res = getResponse(ints, intents)
#         ChatBox.insert(END, "Bot: " + res + '\n\n')
#         ChatBox.config(state=DISABLED)
#         ChatBox.yview(END)
# root = Tk()
# root.title("Chatbot")
# root.geometry("400x500")
# root.resizable(width=FALSE, height=FALSE)
# #Create Chat window
# ChatBox = Text(root, bd=0, bg="white", height="8", width="50", font="Arial",)
# ChatBox.config(state=DISABLED)
# #Bind scrollbar to Chat window
# scrollbar = Scrollbar(root, command=ChatBox.yview, cursor="heart")
# ChatBox['yscrollcommand'] = scrollbar.set
# #Create Button to send message
# SendButton = Button(root, font=("Verdana",12,'bold'), text="Send", width="12", height=5,
#                     bd=0, bg="#f9a602", activebackground="#3c9d9b",fg='#000000',
#                     command= send )
# #Create the box to enter message
# EntryBox = Text(root, bd=0, bg="white",width="29", height="5", font="Arial")
# #EntryBox.bind("<Return>", send)
# #Place all components on the screen
# scrollbar.place(x=376,y=6, height=386)
# ChatBox.place(x=6,y=6, height=386, width=370)
# EntryBox.place(x=128, y=401, height=90, width=265)
# SendButton.place(x=6, y=401, height=90)
# root.mainloop()

In [13]:
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
# from keras.optimizers import SGD
from tensorflow.keras.optimizers import SGD
import random

import nltk
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
import json
import pickle

words=[]
classes = []
documents = []
ignore_letters = ['!', '?', ',', '.']
intents_file = open('/content/drive/MyDrive/intents.json').read()
intents = json.loads(intents_file)

for intent in intents['intents']:
    for pattern in intent['patterns']:
        #tokenize each word
        word = nltk.word_tokenize(pattern)
        words.extend(word)
        #add documents in the corpus
        documents.append((word, intent['tag']))
        # add to our classes list
        if intent['tag'] not in classes:
            classes.append(intent['tag'])
print(documents)
# lemmaztize and lower each word and remove duplicates
words = [lemmatizer.lemmatize(w.lower()) for w in words if w not in ignore_letters]
words = sorted(list(set(words)))
# sort classes
classes = sorted(list(set(classes)))
# documents = combination between patterns and intents
print (len(documents), "documents")
# classes = intents
print (len(classes), "classes", classes)
# words = all words, vocabulary
print (len(words), "unique lemmatized words", words)

pickle.dump(words,open('words.pkl','wb'))
pickle.dump(classes,open('classes.pkl','wb'))

# create our training data
training = []
# create an empty array for our output
output_empty = [0] * len(classes)
# training set, bag of words for each sentence
for doc in documents:
    # initialize our bag of words
    bag = []
    # list of tokenized words for the pattern
    pattern_words = doc[0]
    # lemmatize each word - create base word, in attempt to represent related words
    pattern_words = [lemmatizer.lemmatize(word.lower()) for word in pattern_words]
    # create our bag of words array with 1, if word match found in current pattern
    for word in words:
        bag.append(1) if word in pattern_words else bag.append(0)

    # output is a '0' for each tag and '1' for current tag (for each pattern)
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1
    if len(bag) < len(output_row):
        bag += [0] * (len(output_row) - len(bag))
    elif len(output_row) < len(bag):
        output_row += [0] * (len(bag) - len(output_row))
    training.append([bag, output_row])
# shuffle our features and turn into np.array
random.shuffle(training)
training = np.array(training)
# create train and test lists. X - patterns, Y - intents
train_x = list(training[:,0])
train_y = list(training[:,1])
print("Training data created")

# Create model - 3 layers. First layer 128 neurons, second layer 64 neurons and 3rd output layer contains number of neurons
# equal to number of intents to predict output intent with softmax
model = Sequential()
model.add(Dense(128, input_shape=(len(train_x[0]),), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation='softmax'))

# Compile model. Stochastic gradient descent with Nesterov accelerated gradient gives good results for this model
sgd = SGD(learning_rate=0.01, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

#fitting and saving the model
hist = model.fit(np.array(train_x), np.array(train_y), epochs=172, batch_size=5, verbose=1)
model.save('chatbot_model.h5', hist)

print("model created")

[(['hi'], 'greeting'), (['hello'], 'greeting'), (['whats', 'up'], 'greeting'), (['sup'], 'greeting'), (['is', 'anyone', 'there'], 'greeting'), (['whats', 'good'], 'greeting'), (['hey'], 'greeting'), (['bye'], 'goodbye'), (['cya'], 'goodbye'), (['see', 'you', 'later'], 'goodbye'), (['goodbye'], 'goodbye'), (['im', 'leaving'], 'goodbye'), (['have', 'a', 'good', 'day'], 'goodbye'), (['how', 'old', 'are', 'you'], 'age'), (['what', 'is', 'your', 'age'], 'age'), (['thanks'], 'thanks'), (['thank', 'you'], 'thanks'), (['thankyou'], 'thanks'), (['ty'], 'thanks'), (['I', 'owe', 'you', 'one'], 'thanks'), (['whats', 'is', 'your', 'name'], 'name'), (['whats', 'your', 'name'], 'name'), (['whats', 'should', 'I', 'call', 'you'], 'name'), (['how', 'should', 'I', 'address', 'you'], 'name'), (['no'], 'sky_net_no'), (['nah'], 'sky_net_no'), (['not', 'really'], 'sky_net_no'), (['thats', 'scary'], 'sky_net_no'), (['singularity'], 'sky_net_no'), (['how', 'are', 'you'], 'how_are_you'), (['how', 'are', 'you', 

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


20/20 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.0128 - loss: 4.6885    
Epoch 2/172
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.1936 - loss: 4.5248      
Epoch 3/172
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.2122 - loss: 4.2609  
Epoch 4/172
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.1503 - loss: 3.9514
Epoch 5/172
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.2216 - loss: 3.4412 
Epoch 6/172
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.1414 - loss: 3.2337 
Epoch 7/172
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.1669 - loss: 3.0709
Epoch 8/172
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.2287 - loss: 2.8535
Epoch 9/172
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.2709 - loss: 2.7204 
Epoch 10/172
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.2030 - loss: 2.5897
Epoch 11/172
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.1922 - loss: 2.5253
Epoch 12/172
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accura

model created


In [14]:
!pip install ibm_watson

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 400.1/400.1 kB 6.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.4/69.4 kB 5.0 MB/s eta 0:00:00
  Created wheel for ibm_watson: filename=ibm_watson-8.1.0-py3-none-any.whl size=402518 sha256=56dc9b662be877eb819405693a5e44f92aaab772662353558a352ae4e7870e3a
  Stored in directory: /root/.cache/pip/wheels/9c/ec/3d/75dc52e05ee4b84284f9f481ccbb85260985823c6c7083bf16
Successfully built ibm_watson


In [15]:
import anvil.server

anvil.server.connect("server_ACQH7UGAX7FKFZYLBI3THIFY-VCC5GNK5MLX2QFVD")


In [16]:
import json
from ibm_watson import NaturalLanguageUnderstandingV1
from ibm_cloud_sdk_core.authenticators import IAMAuthenticator
from ibm_watson.natural_language_understanding_v1 \
    import Features, EmotionOptions

@anvil.server.callable
def responsed(msg1):
    global msg
    msg.append(msg1)
    ints = predict_class(msg1)
    res = getResponse(ints, intents)
    return res

@anvil.server.callable
def song_emotion():
    authenticator = IAMAuthenticator('kVJ8aTLAf-GB0AYOP8Y3kN1-PlP_qPdTyAUVhDSCmExR')
    natural_language_understanding = NaturalLanguageUnderstandingV1(
    version='2022-04-07', authenticator=authenticator)

    natural_language_understanding.set_service_url('https://api.au-syd.natural-language-understanding.watson.cloud.ibm.com/instances/38d89b71-adff-434e-9483-3a9c5bd65d7c')
    # text = ""
    # for i in msg:
    #     text = text+i
    text = ""
    len1 = len(msg)

    text = msg[len1-1]+" "+msg[len1-2]+" "+msg[len1-3]+" "+msg[len1-4]+" "+msg[len1-5]
    tone_analysis = natural_language_understanding.analyze(
        text=text,
        features=Features(emotion=EmotionOptions(document=True)),language='en'
    ).get_result()

    # Return the emotion of the text


    emotion = tone_analysis['emotion']['document']['emotion']

    return emotion









In [17]:

print("Chatbot : Hey there, Wassup ?")
# responded function takes text of user and returns chatbot output
msg=[]
for i in range(5):
    m = input("User : ")
    res = responsed(m)
    print("Chatbot : "+res)

ans = song_emotion()
ans_str = json.dumps(ans)
emotion = max(ans,key=ans.get)
print("Emotion : "+emotion)





Chatbot : Hey there, Wassup ?
User : hello
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 74ms/step
Chatbot : Hi there
User : how are you
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
Chatbot : I'm always great. How are you?
User : i am good
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
Chatbot : Thats great to know
User : got some jokes
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
Chatbot : I ate a clock yesterday, it was very time-consuming.
User : oh thats awful
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
Chatbot : Hate to hear it.
Emotion : sadness


In [18]:
!pip install requests

In [19]:
!pip install spotipy


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 261.4/261.4 kB 6.8 MB/s eta 0:00:00


In [35]:
import requests
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
from textblob import TextBlob

# Authenticate with Spotify API
client_credentials_manager = SpotifyClientCredentials(client_id='d10ce52e406b4e34958672456f20cb97', client_secret='11595a2b414f480aba00004b4f9d2276')
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

ans=song_emotion()
ans_str = json.dumps(ans)
print("Emotion : "+emotion)
if ans['joy'] > 0.5:
    mood = 'happy'
elif ans['sadness'] > 0.5:
    mood = 'sad'
elif ans['disgust'] > 0.5:
    mood = 'disgust'
elif ans['anger'] > 0.5:
    mood = 'anger'
elif ans['fear'] > 0.5:
    mood = 'fear'
else:
    mood = 'neutral'
genre_mapping = {
    'happy': 'pop',
    'sad': 'sad',
    'disgust': 'pop',  # Choose a default genre if there's no direct mapping
    'anger': 'rock',
    'fear': 'ambient',
    'neutral': 'chill'
}
seed_genre = genre_mapping.get(mood, 'pop')  # Default to 'pop' if mood not in mapping
results = sp.recommendations(seed_genres=[seed_genre], limit=10)


# Print out song recommendations
print("Recommended Songs:")
for track in results['tracks']:
    print(f"{track['name']} by {track['artists'][0]['name']}")
    print("Spotify URL:", track['external_urls']['spotify'])
    print()




Emotion : sadness
Recommended Songs:
Glory Box by Portishead
Spotify URL: https://open.spotify.com/track/68oUQRwGJGExtkpaSvSbgb

Changes by Faul & Wad
Spotify URL: https://open.spotify.com/track/13qqdlSeF8FcxsRyapDMZ0

Wrong Way by Sublime
Spotify URL: https://open.spotify.com/track/1kWja6NVaLZThMop9MYVDz

You Need Me, I Don't Need You by Ed Sheeran
Spotify URL: https://open.spotify.com/track/5KL4iZkCTZyXl7KnHgfVDj

Let's Get It On by Marvin Gaye
Spotify URL: https://open.spotify.com/track/4Edtgs4Io5HYYN6MDZ4i5G

Clocks by Coldplay
Spotify URL: https://open.spotify.com/track/0BCPKOYdS2jbQ8iyB56Zns

National Anthem by Lana Del Rey
Spotify URL: https://open.spotify.com/track/1M0g1beKC4H9gbrOiSayHW

I'd Rather Dance With You by Kings of Convenience
Spotify URL: https://open.spotify.com/track/4hGOhC87bn7JBdYykOJJvJ

Younger - Kygo Remix by Seinabo Sey
Spotify URL: https://open.spotify.com/track/7hrDA2n8BVGLyHra0rBiFd

Miss You by trentemøller
Spotify URL: https://open.spotify.com/track/1ph

In [24]:
import requests
import base64

# Spotify API Credentials
CLIENT_ID = 'd10ce52e406b4e34958672456f20cb97'
CLIENT_SECRET = '11595a2b414f480aba00004b4f9d2276'

# Function to get access token
def get_access_token():
    # Encode client credentials
    credentials = f"{CLIENT_ID}:{CLIENT_SECRET}"
    encoded_credentials = base64.b64encode(credentials.encode()).decode()

    # Request access token
    headers = {
        'Authorization': f'Basic {encoded_credentials}',
        'Content-Type': 'application/x-www-form-urlencoded'
    }
    data = {
        'grant_type': 'client_credentials'
    }
    response = requests.post('https://accounts.spotify.com/api/token', headers=headers, data=data)

    if response.status_code != 200:
        raise Exception("Failed to authenticate with Spotify API")

    access_token = response.json()['access_token']
    return access_token

# Function to get song recommendations based on emotions
def get_recommendations(access_token, valence, energy):
    url = 'https://api.spotify.com/v1/recommendations'

    params = {
        'seed_genres': 'pop',  # Adjust this to your desired genre
        'min_valence': valence,  # Adjust based on the emotion (0.0 - 1.0)
        'min_energy': energy,    # Adjust based on the emotion (0.0 - 1.0)
        'limit': 10              # Number of recommendations
    }


    headers = {
        'Authorization': f'Bearer {access_token}'
    }
    full_url = f"{url}?{'&'.join([f'{key}={value}' for key, value in params.items()])}"

    response = requests.get(url, headers=headers, params=params)

    if response.status_code != 200:
        raise Exception("Failed to fetch recommendations")

    recommendations = response.json()
    return full_url, recommendations['tracks']


if __name__ == '__main__':
    valence = 0.7  # For positive emotions
    energy = 0.7   # For energetic songs

    try:
        access_token = get_access_token()
        full_url, recommendations = get_recommendations(access_token, valence, energy)

        print("Request URL for Recommendations:", full_url)
        print("\nRecommended Songs:")
        for track in recommendations:
            print(f"{track['name']} by {', '.join([artist['name'] for artist in track['artists']])}")

    except Exception as e:
        print(f"Error: {e}")

Request URL for Recommendations: https://api.spotify.com/v1/recommendations?seed_genres=pop&min_valence=0.7&min_energy=0.7&limit=10

Recommended Songs:
Shotgun by George Ezra
Bang Bang by Jessie J, Ariana Grande, Nicki Minaj
No Money by Galantis
Drowning (feat. Kodak Black) by A Boogie Wit da Hoodie, Kodak Black
Whistle by Flo Rida
Kissing Strangers by DNCE, Nicki Minaj
Let Me Live (feat. Anne-Marie & Mr Eazi) by Rudimental, Major Lazer, Anne-Marie, Mr Eazi
Clandestino by Shakira, Maluma
I Took A Pill In Ibiza - Seeb Remix by Mike Posner, Seeb
DUELE EL CORAZON (feat. Wisin) by Enrique Iglesias, Wisin
